In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Mon Mar 17 16:19:56 2025

@author: user
"""

# =============================================================================
# 
import os
import netCDF4 as nc
import pandas as pd
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import cartopy.crs as ccrs
import netCDF4 as nc
from netCDF4 import Dataset
import shapefile
import time
# =============================================================================
import warnings
warnings.filterwarnings("ignore")  # 忽略所有警告


import shapefile
# from wrf import getvar, ALL_TIMES
from datetime import datetime, timedelta
import cmaps
import sys
# 将目标模块所在的路径添加到 sys.path 中
sys.path.append(r"/home/yfdong/data/work/LF-SAM/code/Library")
from MeteoVarPlot import draw_2DVAR, draw_Bias, draw_VARtemporal, draw_Bias_subplot,draw_2DVAR_subplot ,draw_2DuvVAR ,draw_uvBias_subplot ,draw_2DuvVAR_subplot
from MeteoChartPlot import add_TickGrid
from PreprocessVar import (
    get_MONTH_abbr, get_pentad, get_letter,
    preprocess_var, read_var,Events_configs,
    Read_VarsFromVarsDict, experiments_types,return_Event_Date,
    SubsetDomainConfigure ,create_domain_mask,get_letter,
    get_CoordPair,
    # load_geodata 
)
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
def extract_SubsetRegion_mask_3d(var ,domain_mask):
    # print(var[10,10,10])
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[:,row_start:row_end+1, col_start:col_end+1]
    return sub_var
# --------------------------------- 超参数 ----------------------------------
# 选择输出文件时间间隔
Timedelta=1
# 设置IFdiff
IFdiffs =[True, False]
IFdiffs =[True]
LSMvarList = ['RUNSF', 'RUNSB','rh2','Q2','T2', 'ET', "td2", 'SH2O_MEAN', 'RAINNC', "TSLB_Layer1","FIRA", "FSA", "LH", "SH",] 
ATMvarList = ['height_850hPa' ,"QVAPOR_850hPa","tk_850hPa", 'ua_850hPa' ,'va_850hPa', 'cloudfrac_Low', 'LCL', 'CAPE', 'PBLH', 
              "height_850hPa","rh_850hPa"]
ReadVarList = ATMvarList+LSMvarList
# 定义需要提取的变量列表
def return_var_unit(var_name):
    var_unit_dict = {
        'SM': 'm³/m³',
        'ET': '10⁻² mm/hour',
        'RH_2m': '%',
        "Td_2m" : 'K',
        'RH_850hPa': '%',
        'LCF': '%',
        'LCL': 'm',
        'Prec': 'mm/hour',
        'RUNSF': 'mm/day',
        'RUNSB': 'mm/day',
        'Q850hPa': 'g/kg',
        'Q2m': 'g/kg',
        'ST': 'K',
        'NLW': 'W/m2',
        'NSW': 'W/m2',
        'LH': 'W/m2',
        'SH': 'W/m2',
        'CAPE': 'J/kg',
        'PBLH': 'm',
        'T2m': 'K',
        'T850hPa': 'K'
    }
    return var_unit_dict.get(var_name, 'Unknown Unit')

def return_var_abbreviation(var_name):
    var_abbreviation_dict = {
        'SH2O_MEAN': 'SM',
        # 'SH2O_Layer1': 'SM',
        'ET': 'ET',
        'rh2': 'RH_2m',
        'td2': 'Td_2m',
        'rh_850hPa': 'RH_850hPa',
        'cloudfrac_Low': 'LCF',
        'LCL': 'LCL',
        'RAINNC': 'Prec',
        'RUNSF': 'RUNSF',
        'RUNSB': 'RUNSB',
        'QVAPOR_850hPa': 'Q850hPa',
        'Q2': 'Q2m',
        'TSLB_Layer1': 'ST',
        'FIRA': 'NLW',
        'FSA': 'NSW',
        'LH': 'LH',
        'SH': 'SH',
        'CAPE': 'CAPE',
        'PBLH': 'PBLH',
        'T2': 'T2m',
        'tk_850hPa': 'T850hPa'
    }
    return var_abbreviation_dict.get(var_name, 'Unknown Var')

def return_var_fullname(var_name):
    var_fullname_dict = {
        'SM': 'Column-integrated average soil moisture',
        'ET': 'Evaporation',
        'RH_2m': 'Relative humidity at 2 m',
        'Td_2m': 'Dew point temperature at 2 m',
        'RH_850hPa': 'Relative humidity at 850 hPa',
        'LCF': 'Low level cloud fraction',
        'LCL': 'Lifted Condensation Level',
        'Prec': 'Precipitation rate',
        'RUNSF': 'Surface runoff',
        'RUNSB': 'Subsurface runoff',
        'Q850hPa': 'Water vapor mixing ratio at 850 hPa',
        'Q2m': 'Water vapor mixing ratio at 2 m',
        'ST': 'Soil temperature (0~10 cm)',
        'NLW': 'Net longwave radiation',
        'NSW': 'Net shortwave radiation',
        'LH': 'Latent heat flux',
        'SH': 'Sensible heat flux',
        'CAPE': 'Convective Available Potential Energy',
        'PBLH': 'Planetary boundary layer height',
        'T2m': 'Temperature at 2 m',
        'T850hPa': 'Temperature at 850 hPa'
    }
    return var_fullname_dict.get(var_name, 'Unknown Var')
# =============================================================================
# custom Functions:
import matplotlib.dates as mdates
BasePath = "/home/yfdong/data/work/LF-SAM/code/Library"
DataPath = '/raid61/yfdong/data/work/LF-SAM/output'
def load_geodata(GeoPath ,IF_LandMask):
    """Load geographic data from NetCDF file"""
    with Dataset(GeoPath) as geo_nc:
        lat = geo_nc.variables['XLAT_M'][0,:,:]
        lon = geo_nc.variables['XLONG_M'][0,:,:]
        lu_index = geo_nc.variables['LU_INDEX'][0,:]
        landmask = geo_nc.variables['LANDMASK'][0,:]
        dem = geo_nc.variables['HGT_M'][0,:]
    if IF_LandMask: #只mask掉海洋，保留湖泊
        lu_index[lu_index==16]=np.nan # mask掉海洋
        # 将其他非 NaN 值赋值为 1
        lu_index[~np.isnan(lu_index)] = 1
        Landmask = lu_index.copy()
    else: # mask全部水体
        Landmask = landmask.copy()
    return lat, lon, Landmask, dem
# 读取地形数据
LAT, LON, LandMask, DEM = load_geodata(GeoPath ="/raid61/yfdong/data/work/LF-SAM/Domain/def/geo_em.nc" ,IF_LandMask=False)
LandMask[LandMask==0]=np.nan
LandMask_3d = LandMask[np.newaxis, :, :]

# 读取区域设置
SubsetRegion = SubsetDomainConfigure[0] #黄山
DomainRegion = [25,34,110,123] # 绘图范围#[27,33,114.5,122.5] #SubsetRegion["DomainRegion"]
DomainRegion = [24.5,35,109,123] # 绘图范围#
DomainRegion2 = [24,35,110,123] # 提取mask范围
DomainName = SubsetRegion["DomainName"]
DomainMask = create_domain_mask(LAT, LON, DomainRegion2)

CalibrationStatus = "def" # "def" ,"calib"、
experiment = experiments_types[0]
LAYER =1
# EventName = Event['EventName']
# FIG_PATH = f'/home/yfdong/data/work/LF-SAM/output/Event/{EventName}/fig'

subsample = 2
subLAT =extract_SubsetRegion_mask(LAT ,DomainMask)[ ::subsample, ::subsample]  
subLON =extract_SubsetRegion_mask(LON ,DomainMask)[ ::subsample, ::subsample]  
subDEM =extract_SubsetRegion_mask(DEM ,DomainMask)[ ::subsample, ::subsample] 


EventState = 'Dur'
FeedbackState = 'Positive'
FeedbackState = 'Negative'
import pandas as pd
Event_df = pd.read_excel(f'/home/yfdong/data/work/LF-SAM/code/AnalysisModule/HuangshanPrecAnalysis/EventDateDf_{FeedbackState}.xlsx')
Event_subset_df = Event_df.copy()
EventNum = len(Event_subset_df)

In [ ]:
print(f"FeedbackState: {FeedbackState}")

In [ ]:
cache_filename = (
    f"WRF_cache_SyntheticAnalysis_{FeedbackState}_{EventState}_{CalibrationStatus}_"
    f"{Timedelta}h_{EventNum}events.nc"
)
cache_path = os.path.join(DataPath, 'Event/HeavyRainfall', 'cache', cache_filename)
if os.path.exists(cache_path):
    print(f"读取缓存文件: {cache_path}")
    with nc.Dataset(cache_path, 'r') as ds:
        # 读取结果数据
        results = {}

        # 初始化Ctrl和Expl的字典
        results['Ctrl'] = {}
        results['Expl'] = {}

        # 遍历ReadVarList中的每个变量
        for var in ReadVarList:
            abbr = return_var_abbreviation(var)     # 获取标准缩写
            if var in LSMvarList:
                # 读取控制组（Ctrl）的数据
                results['Ctrl'][abbr] = ds[f'Ctrl_{var}'][:] * LandMask_3d[ :, ::subsample, ::subsample] 
                # 读取实验组（Expl）的数据
                results['Expl'][abbr] = ds[f'Expl_{var}'][:] * LandMask_3d[ :, ::subsample, ::subsample] 
                if var in ['ET']:
                    results['Ctrl'][abbr] = results['Ctrl'][abbr]*(1/24)*100 # 转化单位 mm/day -> 0.01 * mm/hour
                    results['Expl'][abbr] = results['Expl'][abbr]*(1/24)*100
                elif var in ['cloudfrac_Low']:
                    results['Ctrl'][abbr] = results['Ctrl'][abbr]*100 # 转化单位 - -> %
                    results['Expl'][abbr] = results['Expl'][abbr]*100
            elif var in ATMvarList:
                # 读取控制组（Ctrl）的数据
                results['Ctrl'][abbr] = ds[f'Ctrl_{var}'][:] 
                # 读取实验组（Expl）的数据
                results['Expl'][abbr] = ds[f'Expl_{var}'][:] 
            else:
                print(f"未知的变量: {var}")
                
else:
    print("未找到缓存文件，重新处理数据...")

In [ ]:
plt.imshow(results['Expl']['Q2m'][10,:])

In [ ]:
print(results['Expl']['Q2m'][:].shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def draw_LSMvar_boxplot(CtrbResults, WaterCycleVarListAbb, EnergyCycleVarListAbb, 
                        axs, IDx, 
                        plotMax1, plotMax2, Title):

    # ------------------------ 水分循环 ------------------------
    # 将数据转换为适合绘图的 DataFrame 格式
    data_for_plot = []
    for var in WaterCycleVarListAbb:
        # print(var)
        # 提取每个变量的 CRE 数据
        rc_lf_values = CtrbResults['CRE'][var].flatten()  # 展平数组
        filtered_values = rc_lf_values[~np.isnan(rc_lf_values)]
        for value in filtered_values:
            data_for_plot.append({'Variable': var, 'CRE': value})
    # 创建 DataFrame
    df_plot = pd.DataFrame(data_for_plot)
    sampled_df = df_plot.sample(frac=0.1, random_state=42)  # 设置 random_state 以保证每次运行时结果一致

    ax = axs[IDx, 0]  
    # 绘制箱型图
    sns.boxplot(data=df_plot, x='Variable', y='CRE', palette='Set2',
                    showmeans=True,
                    linewidth=1.5, 
                    width=0.6,
                    saturation=0.6,
                    meanprops={
                        'marker': 'o',
                        'markerfacecolor': 'white', 
                        'markeredgecolor': 'black',
                        'markersize': 5
                    },
                    
                    showfliers=True,
                    flierprops={
                    'marker': 'o',          # 设置异常点的形状
                    'markerfacecolor': 'grey',  # 设置异常点的填充颜色
                    'markeredgecolor': 'black',# 设置异常点的边框颜色
                    'markersize': 1,        # 设置异常点的大小
                    'alpha': 0.05            # 设置异常点的透明度
                    },
                    ax = axs[IDx, 0]  ,
                    # boxprops=dict(linewidth=1.5, edgecolor='#696969'),  # 这里设置了整个箱子边缘的颜色
                    )  # 使用 Set2 调色板
    ax.text(0.025, 0.95, f"({get_letter(IDx*2+0)})", fontsize=14, transform=ax.transAxes, va='top')    
    # 添加标题和标签
    # plt.title('Boxplot of CRE for Each Variable', fontsize=14)
    ax.set_xlabel(None, fontsize=12)
    ax.set_ylabel(Title, fontsize=12)
    ax.set_ylim(-plotMax1 , plotMax1)
    # 在 y=0 处添加红色虚线
    ax.axhline(y=0, color='#8B2323', linestyle='--', linewidth=1.)
    if IDx == 1:
        ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='center')
    else:
        ax.set_xticklabels([])
        
    if IDx == 0:
        ax.set_title('Water blance' )
    add_TickGrid(ax, IFminorOn=True)
    # ax.text(-0.025, 0.5, Title, fontsize=14, transform=ax.transAxes, va='center', ha='right')
    # ----------------------- 能量循环 --------------------------
    ax = axs[IDx, 1]
    data_for_plot = []
    for var in EnergyCycleVarListAbb:
        
        # 提取每个变量的 CRE 数据
        rc_lf_values = CtrbResults['CRE'][var].flatten()  # 展平数组
        filtered_values = rc_lf_values[~np.isnan(rc_lf_values)]
        for value in filtered_values:
            data_for_plot.append({'Variable': var, 'CRE': value})

    # 创建 DataFrame
    df_plot = pd.DataFrame(data_for_plot)

    sns.boxplot(data=df_plot, x='Variable', y='CRE', palette='Set1',
                    showmeans=True,
                    linewidth=1.5, 
                    width=0.6,
                    saturation=0.6,
                    meanprops={
                        'marker': 'o',
                        'markerfacecolor': 'white', 
                        'markeredgecolor': 'black',
                        'markersize': 5
                    },
                    showfliers=True,
                    flierprops={
                    'marker': 'o',          # 设置异常点的形状
                    'markerfacecolor': 'grey',  # 设置异常点的填充颜色
                    'markeredgecolor': 'black',# 设置异常点的边框颜色
                    'markersize': 1,        # 设置异常点的大小
                    'alpha': 0.05            # 设置异常点的透明度
                    },
                    ax = axs[IDx, 1],
                    # boxprops=dict(linewidth=1.5, edgecolor='#696969'),  # 这里设置了整个箱子边缘的颜色
                    )  # 使用 Set2 调色板
    ax.text(0.025, 0.95, f"({get_letter(IDx*2+1)})", fontsize=14, transform=ax.transAxes, va='top')    
    # 添加标题和标签
    # plt.title('Boxplot of CRE for Each Variable', fontsize=14)
    ax.set_xlabel(None, fontsize=12)
    ax.set_ylabel(None, fontsize=12)
    ax.set_ylim(-plotMax2, plotMax2)
    # 在 y=0 处添加红色虚线
    ax.axhline(y=0, color='#8B2323', linestyle='--', linewidth=1.)
    # 旋转横坐标标签（避免重叠） 
    
    if IDx == 1:
        ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='center')
    else:
        ax.set_xticklabels([])
        
    if IDx == 0:
        ax.set_title('Energy blance' )
    add_TickGrid(ax, IFminorOn=True)

    pass

def QuantifyLSMvarContributions_toExcel(CtrbResults, TotalVarList, ExcelFileName):
    # 初始化 DataFrame
    LSMvar_df = pd.DataFrame({
        'Variable': [],
        'Unit':[],
        'Mean':[],
        'Diff': [],
        'HMS': [], # HMS: Hydrometeorological Sensitivity of Lateral Flow
        'RD(%)': [], # RD: Relative difference
        'CRE(%)': [], # CRE: Coupling Relative Effect of Lateral Flow
    })
    # 遍历 WaterCycleVarList 中的每个变量
    for var in TotalVarList:
        # print(var)
        VarFullname = return_var_fullname(var)
        # 提取每个变量的 CRE 数据
        rc_lf_values = CtrbResults['CRE'][var].flatten()  # 展平数组
        diff_values = CtrbResults['Diff'][var].flatten() 
        # RD_values = CtrbResults['RD'][var].flatten() 
        hms_lf_values = CtrbResults['HMS'][var].flatten()
        # 设置保留小数位数
        NP_ROUND = 4
        # 计算均值和标准差
        Ctrl_mean = np.round( np.nanmean(CtrbResults['Ctrl'][var]) ,NP_ROUND)
        rc_lf_mean = np.round( np.nanmean(rc_lf_values) ,NP_ROUND)
        rc_lf_std = np.round( np.nanstd(rc_lf_values) ,NP_ROUND)
        diff_mean = np.round( np.nanmean(diff_values) ,NP_ROUND)
        # RD_mean = np.round( np.nanmean(RD_values) ,4)
        RD_mean = np.round(diff_mean/Ctrl_mean * 100, NP_ROUND)
        diff_std = np.round( np.nanstd(diff_values) ,NP_ROUND)
        hms_lf_mean = np.round(np.nanmean(hms_lf_values), NP_ROUND)
        # 创建新的 DataFrame 行
        new_row = pd.DataFrame({
            'Variable': [f"{VarFullname} ({var})"],
            'Unit': [return_var_unit(var)],
            'Mean': [Ctrl_mean],
            'Diff': [diff_mean],
            'HMS': [hms_lf_mean],
            'RD(%)': [RD_mean],
            'CRE(%)': [rc_lf_mean],
        })
        
        # 将新行添加到 LSMvar_df 中
        LSMvar_df = pd.concat([LSMvar_df, new_row], ignore_index=True)
    # 打印 DataFrame
    # print(LSMvar_df)
    LSMvar_df.to_excel(ExcelFileName)
    

    return LSMvar_df

def calculate_CRE_LF(ReadVarList, results, DomainMask):
    CtrbResults = {
    'Ctrl': {},
    'Expl':{},
    'Diff': {},
    'RD': {},
    'CRE': {}, # 相对贡献 ：
    'HMS':{},
    }
    DomainMask_3d = DomainMask.expand_dims('new_dim')
    # 将变量区域化，评估增加侧向流之后，变量波动相较于该区域控制实验的变量振幅
    for var in ReadVarList:
        # print(var)
        if var=='RAINNC':
            CtrbResults['Ctrl'][var] = extract_SubsetRegion_mask_3d(results['Ctrl'][var], DomainMask[::subsample, ::subsample]) *24 # mm/hour -> mm/day
            CtrbResults['Diff'][var] = extract_SubsetRegion_mask_3d(results['Expl'][var] - results['Ctrl'][var] ,DomainMask[::subsample, ::subsample]) *24 # mm/hour -> mm/day
        else:
            CtrbResults['Ctrl'][var] = extract_SubsetRegion_mask_3d(results['Ctrl'][var], DomainMask[::subsample, ::subsample]) 
            CtrbResults['Diff'][var] = extract_SubsetRegion_mask_3d(results['Expl'][var] - results['Ctrl'][var] ,DomainMask[::subsample, ::subsample]) 
            
    for var in ReadVarList:
        CtrlVarStd = np.nanstd(CtrbResults['Ctrl'][var])
        CtrbResults['HMS'][var] = np.nanmean( CtrbResults['Diff'][var])/ np.nanmean(CtrbResults['Diff']['SM']) *0.05
        CtrbResults['CRE'][var] = np.nanmean( CtrbResults['Diff'][var] ,axis=0)/ np.nanmean(CtrlVarStd  ,axis=0) *100#CRE:Coupling Relative Effect of Lateral Flow

    return CtrbResults

In [ ]:
# ----------------------------------------------- 计算模块 ----------------------------------------
WaterCycleVarListAbb = [ 'SM',  'Prec', 'ET',  'Q2m', "Q850hPa", 'LCF', 'LCL', #'RAINNC', "QVAPOR_850hPa", 'Q2', 'RH_2m', 'QVAPOR_2m'
            ] #
EnergyCycleVarListAbb = [ 
            "ST", "NLW", "NSW", "LH", "SH",'CAPE', 'PBLH', 'T2m', 'T850hPa' #'T2',  "tk_850hPa",FIRA", "FSA", "LH", "SH",'CAPE', 'PBLH',  #'T2',  "tk_850hPa",
            ] #
ReadVarListAbb = WaterCycleVarListAbb + EnergyCycleVarListAbb
DomainRegion2 = [26.5, 28.5, 114.5, 117.5]   # 吉泰盆地
# DomainRegion2 = [27, 28.7, 114.5, 117.5]   # 吉泰盆地
DomainMaskJTB = create_domain_mask(LAT, LON, DomainRegion2)
CtrbResultsJTB = calculate_CRE_LF(ReadVarListAbb, results, DomainMaskJTB)

DomainRegion2 = [28.7,30.7,116.8,119.8] # 皖南山区
DomainMaskSAM = create_domain_mask(LAT, LON, DomainRegion2)
CtrbResultsSAM = calculate_CRE_LF(ReadVarListAbb, results, DomainMaskSAM)


In [ ]:
ReadVarListAbb

In [ ]:
plt.rcParams['axes.edgecolor'] = 'black'  # 边框颜色
plt.rcParams['axes.linewidth'] = 0.5     # 边框线宽
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10*1., 4*1.*2/3*2),
                        gridspec_kw={'width_ratios': [len(WaterCycleVarListAbb), len(EnergyCycleVarListAbb)]}, 
                        dpi=500,
                        sharex=False,
                        sharey=False
                )

draw_LSMvar_boxplot(CtrbResultsJTB, WaterCycleVarListAbb, EnergyCycleVarListAbb, 
                    axs, IDx=0, 
                    plotMax1=40, plotMax2=30, Title='JTB Region')
draw_LSMvar_boxplot(CtrbResultsSAM, WaterCycleVarListAbb, EnergyCycleVarListAbb, 
                    axs, IDx=1, 
                    plotMax1=40, plotMax2=30, Title='SAM Region')

# 显示图像
plt.tight_layout()  # 自动调整布局
plt.show() 

In [ ]:
CtrbResultsJTB
QuantifyLSMvarContributions_toExcel(CtrbResultsJTB, ReadVarListAbb, 
                                    ExcelFileName=f"LSMvar_CouplingRelativeEffect_JTB_{FeedbackState}.xlsx")



In [ ]:
CtrbResultsSAM
QuantifyLSMvarContributions_toExcel(CtrbResultsSAM, ReadVarListAbb, 
                                    ExcelFileName=f"LSMvar_CouplingRelativeEffect_SAM_{FeedbackState}.xlsx")
